# Installation

Tun this script in a ubuntu system 

In [ ]:
# Install Python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-dev python3.9-distutils -y
!sudo apt-get install python3.9-venv -y

# Install pip for Python 3.9
!wget https://bootstrap.pypa.io/get-pip.py
!python3.9 get-pip.py

In [ ]:
!source /tmp/tf_venv/bin/activate && python --version
!pip --version

import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'tensorflow'


In [ ]:
%pip install protobuf==3.20.3
%pip install numpy==1.23.5
%pip install tensorflow==2.7

In [ ]:
# Verify the installation
!source /tmp/tf_venv/bin/activate && python -c "import tensorflow as tf; print(tf.__version__)"

# Example of training a model

In [ ]:
# Step 1: Imports
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import os

In [ ]:
# Step 2: Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Resize to match MobileNetV2 input
x_train = tf.image.resize(x_train, (96, 96)) / 255.0
x_test = tf.image.resize(x_test, (96, 96)) / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Step 3: Load MobileNetV2 and add custom head
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(96, 96, 3)))
base_model.trainable = False  # Freeze base

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
# Step 4: Compile and train
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# For a quick run, train on just a few epochs
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))


In [ ]:
# Step 5: Export the trained model
export_path = "model.keras"
model.save(export_path)
print(f"Model saved to: {os.path.abspath(export_path)}")
